Final project restart

In [142]:
import sys
import pickle
sys.path.append("../tools/")

import pandas as pd
import numpy as np

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

import matplotlib.pyplot as plt
%matplotlib inline
from pprint import pprint

from sklearn.preprocessing import Imputer
                                                
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

from sklearn.decomposition import PCA

from sklearn.cross_validation import StratifiedShuffleSplit

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from tester import test_classifier

from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_fscore_support

### Data Import

In [148]:
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pd.DataFrame(data_dict)
df = df.transpose()
df = df.drop('email_address', axis=1)
df = df.astype(float)
df = df.drop('TOTAL')
df = df.drop("THE TRAVEL AGENCY IN THE PARK")
df = df.drop("loan_advances", axis=1)
features_list = list(df.columns)
features_list.remove('poi')
features = df[features_list]
labels = df['poi']

In [149]:
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = \
    cross_validation.train_test_split(features, labels, test_size=0.3, random_state=42)

## Decision Tree Pipeline: median imputed

In [151]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Pipeline = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='median')),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])

pipeline = Pipeline.fit(features_train, labels_train)

print """DecisionTreeClassifier\nImputer: median\nNormalize: StandardScaler\nPCA: dimensions unspecified"""
print "\nAccuracy Score:", pipeline.score(features_test, labels_test)

clf_pipeline_pred = pipeline.predict(features_test)
print "\nRecall:", recall_score(labels_test, clf_pipeline_pred)
print "\nPrecision:", precision_score(labels_test, clf_pipeline_pred)

DecisionTreeClassifier
Imputer: median
Normalize: StandardScaler
PCA: dimensions unspecified

Accuracy Score: 0.863636363636

Recall: 0.6

Precision: 0.428571428571


## New Feature-Data Reimport

In [152]:
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pd.DataFrame(data_dict)
df = df.transpose()
df = df.drop('email_address', axis=1)
df = df.astype(float)
df = df.drop('TOTAL')
df = df.drop("THE TRAVEL AGENCY IN THE PARK")
df = df.drop("loan_advances", axis=1)
#new feature
df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
features_list = list(df.columns)
features_list.remove('poi')
features = df[features_list]
labels = df['poi']

In [153]:
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = \
    cross_validation.train_test_split(features, labels, test_size=0.3, random_state=42)

In [154]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Pipeline = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='median')),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])

pipeline = Pipeline.fit(features_train, labels_train)

print """DecisionTreeClassifier\nImputer: median\nNormalize: StandardScaler\nPCA: dimensions unspecified"""
print "\nAccuracy Score:", pipeline.score(features_test, labels_test)

clf_pipeline_pred = pipeline.predict(features_test)
print "\nRecall:", recall_score(labels_test, clf_pipeline_pred)
print "\nPrecision:", precision_score(labels_test, clf_pipeline_pred)

DecisionTreeClassifier
Imputer: median
Normalize: StandardScaler
PCA: dimensions unspecified

Accuracy Score: 0.795454545455

Recall: 0.4

Precision: 0.25


Ok, so it is the new variable that causes the problem. I will try to convince myself of this by removing the new variable and trying it again. 

In [156]:
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pd.DataFrame(data_dict)
df = df.transpose()
df = df.drop('email_address', axis=1)
df = df.astype(float)
df = df.drop('TOTAL')
df = df.drop("THE TRAVEL AGENCY IN THE PARK")
df = df.drop("loan_advances", axis=1)
#new feature
# df['deferred_ratio'] = df['deferred_income']/(df['total_payments'] + 1)
features_list = list(df.columns)
features_list.remove('poi')
features = df[features_list]
labels = df['poi']

In [157]:
from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = \
    cross_validation.train_test_split(features, labels, test_size=0.3, random_state=42)

In [158]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

Pipeline = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='median')),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])

pipeline = Pipeline.fit(features_train, labels_train)

print """DecisionTreeClassifier\nImputer: median\nNormalize: StandardScaler\nPCA: dimensions unspecified"""
print "\nAccuracy Score:", pipeline.score(features_test, labels_test)

clf_pipeline_pred = pipeline.predict(features_test)
print "\nRecall:", recall_score(labels_test, clf_pipeline_pred)
print "\nPrecision:", precision_score(labels_test, clf_pipeline_pred)

DecisionTreeClassifier
Imputer: median
Normalize: StandardScaler
PCA: dimensions unspecified

Accuracy Score: 0.863636363636

Recall: 0.6

Precision: 0.428571428571


Ok, so now I can get the thing to work. The best thing so far is using pca with no actual reduction in the number of dimensions and no new features added. Now I am going to try to submit that to the udacity grader. 

## Submitting to the Udacity Grader

In [130]:
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
df = pd.DataFrame(data_dict)
df = df.transpose()
df = df.drop('email_address', axis=1)
df = df.astype(float)
df = df.drop('TOTAL')
df = df.drop("THE TRAVEL AGENCY IN THE PARK")
df = df.drop("loan_advances", axis=1)
features_list = list(df.columns)
features_list.remove('poi')
features = df[features_list]
labels = df['poi']

from sklearn import cross_validation
features_train, features_test, labels_train, labels_test = \
    cross_validation.train_test_split(features, labels, test_size=0.3, random_state=42)

In [168]:
from tester import test_classifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

Pipeline = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='median')),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])

pipeline = Pipeline.fit(features_train, labels_train)

print """DecisionTreeClassifier\nImputer: median\nNormalize: StandardScaler\nPCA: dimensions unspecified"""
print "\nAccuracy Score:", pipeline.score(features_test, labels_test)

clf_pipeline_pred = pipeline.predict(features_test)
print "\nRecall:", recall_score(labels_test, clf_pipeline_pred)
print "\nPrecision:", precision_score(labels_test, clf_pipeline_pred)


# x = [x for x in range(2,19)]
# d = [d for d in range(2,10)]
# param_grid = {'pca__n_components': x,
#               'clf__max_depth': d}

# gridCV_object = GridSearchCV(estimator = Pipeline, 
#                              param_grid = param_grid)

# gridCV_object.fit(features_train, labels_train)

# print "Best parameters from the grid search:", pprint(gridCV_object.best_params_)

# clf_gridCV = gridCV_object.best_estimator_

# print "\nBest Estimator Accuracy:", clf_gridCV.score(features_test, labels_test)

# clf_gridCV_pred = clf_gridCV.predict(features_test)

# print "\n\nRecall Score:", recall_score(labels_test, clf_gridCV_pred)
# print "\n\nPrecision Score:", precision_score(labels_test, clf_gridCV_pred)

DecisionTreeClassifier
Imputer: median
Normalize: StandardScaler
PCA: dimensions unspecified

Accuracy Score: 0.863636363636

Recall: 0.6

Precision: 0.428571428571


#### Extra data prep to give to the grader. 
Put 'poi' at the top of the features_list and put the whole pandas data frame into a dictionary called 'data_dict'. 

In [169]:
features_list[0] = 'poi'
df_1 = df.transpose()
data_dict = df_1.to_dict()

#### Same model submitted to the grader: 

In [172]:
from tester import test_classifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA

clf = Pipeline([
        ('imp', Imputer(missing_values='NaN', strategy='median')),
        ('std', StandardScaler()),
        ('pca', PCA()),
        ('clf', DecisionTreeClassifier(random_state = 53))
    ])

print "\n\nAnd these are the results going through the test classifier:\n"
test_classifier(clf, data_dict, features_list, folds = 1000)



And these are the results going through the test classifier:

Pipeline(steps=[('imp', Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)), ('std', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, n_components=None, whiten=False)), ('clf', DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=N...plit=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=53, splitter='best'))])
	Accuracy: 0.79173	Precision: 0.27005	Recall: 0.33000	F1: 0.29703	F2: 0.31597
	Total predictions: 15000	True positives:  660	False positives: 1784	False negatives: 1340	True negatives: 11216

